In [24]:
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neural_network import MLPClassifier  # MLP is an NN
from sklearn import svm
import numpy as np
import argparse
import imutils  # If you are unable to install this library, ask the TA; we only need this in extract_hsv_histogram.
import cv2
import os
import random
import time
from commonfunctions import *
# Depending on library versions on your system, one of the following imports 
from sklearn.model_selection import train_test_split

In [25]:
def eq_hist(img):
    # Step 1: Compute the histogram
    img= img.astype(int)
    h, _ = np.histogram(img, bins=256, range=(0, 256))
    
    # Step 2: Compute the cumulative distribution function (CDF)
    h_c = np.cumsum(h)
    
    # Step 3: Normalize the CDF to the range [0, 255]
    t = np.round(255 * h_c / h_c[-1]).astype(np.uint8)
    
    # Step 4: Map the original pixel values to equalized values
    equalized_img = t[img]
    
    return equalized_img



def gamma_correction(c,gamma,img):
    gamma_img = c*np.power(img,gamma)
    return gamma_img

In [26]:

def detect_hand(image):
    copy = image.copy()
    hsv = cv2.cvtColor(copy, cv2.COLOR_BGR2HSV)

    # Skin color range in HSV
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)

    # Create mask
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Load face detector
    
    # Morphological operations to clean noise
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Filter by area and aspect ratio
        min_area = 5000
        filtered_contours = [c for c in contours if cv2.contourArea(c) > min_area]

        if filtered_contours:
            hand_contour = max(filtered_contours, key=cv2.contourArea)

            # Convex Hull and Convexity Defects
            hull = cv2.convexHull(hand_contour, returnPoints=False)
            defects = cv2.convexityDefects(hand_contour, hull)

            # Draw the hand contour and convex hull
            cv2.drawContours(copy, [hand_contour], -1, (0, 255, 0), 2)
            cv2.drawContours(copy, [cv2.convexHull(hand_contour)], -1, (255, 0, 0), 2)

            # Highlight defects
            if defects is not None:
                for i in range(defects.shape[0]):
                    s, e, f, d = defects[i][0]
                    far = tuple(hand_contour[f][0])
                    cv2.circle(copy, far, 5, (0, 0, 255), -1)

    return copy


In [27]:
import cv2
import numpy as np

def detect_hand(image):
    copy = image.copy()
    hsv = cv2.cvtColor(copy, cv2.COLOR_BGR2HSV)

    # Skin color range in HSV
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)

    # Create mask for skin color
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Morphological operations to clean noise
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Load the face detector (Haar Cascade Classifier)
    face_cascade = cv2.CascadeClassifier(cv2.data.haarcascades + 'haarcascade_frontalface_default.xml')

    # Detect faces in the image
    gray = cv2.cvtColor(copy, cv2.COLOR_BGR2GRAY)
    faces = face_cascade.detectMultiScale(gray, scaleFactor=1.1, minNeighbors=5, minSize=(30, 30))

    # Initialize an empty image for the result
    result = np.zeros_like(image)

    # Find contours for hand detection
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Filter by area and aspect ratio
        min_area = 5000
        filtered_contours = [c for c in contours if cv2.contourArea(c) > min_area]

        if filtered_contours:
            # Loop over the filtered contours and draw bounding boxes
            for contour in filtered_contours:
                # Get bounding rectangle for each contour
                x, y, w, h = cv2.boundingRect(contour)
                # Create a mask for the current rectangle
                hand_mask = np.zeros_like(image)
                cv2.drawContours(hand_mask, [contour], -1, (255, 255, 255), thickness=cv2.FILLED)

                # Apply the hand mask to the original image
                temp_result = cv2.bitwise_and(image, hand_mask)

                # Copy the result to ensure the hand areas are preserved
                result = cv2.add(result, temp_result)

    # Now mask out the face regions by adding black rectangles in the result
    for (x, y, w, h) in faces:
        cv2.rectangle(result, (x, y), (x + w, y + h), (0, 0, 0), -1)

    return result


In [28]:



def highlight_skin_with_two_largest(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define skin color range in HSV
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)

    # Create a mask for skin regions
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Filter and sort contours by area
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 3000]
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    # Create a copy of the original image to color the selected regions
    result = image.copy()

    if len(contours) >= 2:
        # Color the largest contour blue
        cv2.drawContours(result, [contours[0]], -1, (255, 0, 0), -1)  # Blue for the largest
        
        # Color the second largest contour red
        cv2.drawContours(result, [contours[1]], -1, (0, 0, 255), -1)  # Red for the second largest
    elif len(contours) == 1:
        # Split the largest contour into two along its bounding box
        x, y, w, h = cv2.boundingRect(contours[0])
        mid_x = x + w // 2

        # Split the mask of the largest contour
        contour_mask = np.zeros_like(mask)
        cv2.drawContours(contour_mask, [contours[0]], -1, 255, -1)
        left_half = contour_mask[:, :mid_x]
        right_half = contour_mask[:, mid_x:]

        # Color the left half blue
        result[:, :mid_x][left_half > 0] = (255, 0, 0)  # Blue
        
        # Color the right half red
        result[:, mid_x:][right_half > 0] = (0, 0, 255)  # Red

    return result

def detect_hand_using_fingers(image):
    copy = image.copy()
    hsv = cv2.cvtColor(copy, cv2.COLOR_BGR2HSV)

    # Skin color range in HSV
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)

    # Create mask for skin color
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Morphological operations to clean noise
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    mask = cv2.morphologyEx(mask, cv2.MORPH_CLOSE, kernel)

    # Find contours for hand detection
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    if contours:
        # Filter by area and aspect ratio
        min_area = 5000
        filtered_contours = [c for c in contours if cv2.contourArea(c) > min_area]

        if filtered_contours:
            # Get the largest contour (assumed to be the hand)
            hand_contour = max(filtered_contours, key=cv2.contourArea)

            # Approximate the contour to make it simpler (remove small jaggedness)
            epsilon = 0.02 * cv2.arcLength(hand_contour, True)
            approx_contour = cv2.approxPolyDP(hand_contour, epsilon, True)

            # Get convex hull and convexity defects to detect fingers
            hull = cv2.convexHull(approx_contour, returnPoints=False)

            # Ensure the convex hull is valid
            if len(hull) > 3:  # At least 4 points needed for convexity defects
                defects = cv2.convexityDefects(hand_contour, hull)

                # Draw convex hull
                cv2.drawContours(copy, [cv2.convexHull(hand_contour)], -1, (0, 255, 0), 2)

                if defects is not None:
                    # Loop through defects to find finger tips
                    finger_tips = []
                    for i in range(defects.shape[0]):
                        s, e, f, d = defects[i][0]
                        far = tuple(hand_contour[f][0])
                        finger_tips.append(far)

                        # Draw the finger tips
                        cv2.circle(copy, far, 5, (0, 0, 255), -1)

                    # If there are finger tips, use them to define the hand region
                    if finger_tips:
                        # Use finger tips to estimate the upper region of the hand
                        # Find the top-most point among finger tips
                        finger_tips.sort(key=lambda x: x[1])  # Sort by y-coordinate (topmost first)
                        top_finger = finger_tips[0]  # This is the top-most finger tip

                        # Assuming the hand's region is above this finger
                        x, y, w, h = cv2.boundingRect(hand_contour)
                        extended_y = max(y - 50, 0)  # Extend upwards from the palm
                        cv2.rectangle(copy, (x, extended_y), (x + w, y + h), (0, 255, 255), 2)

    return copy


In [29]:

def highlight_thin_skin(image):
    # Convert the image to HSV color space
    hsv = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)

    # Define skin color range in HSV
    lower_skin = (0, 20, 70)
    upper_skin = (20, 255, 255)

    # Create a mask for skin regions
    mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Find contours in the mask
    contours, _ = cv2.findContours(mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)

    # Create a copy of the original image to color the selected regions
    result = image.copy()

    # Iterate over each contour to check aspect ratio dynamically
    for contour in contours:
        if cv2.contourArea(contour)>1000 and cv2.contourArea(contour)<10000:  # Filter small contours
            # Fit a minimum area rotated rectangle around the contour
            rect = cv2.minAreaRect(contour)
            box = cv2.boxPoints(rect)  # Get the 4 points of the rectangle
            box = np.int0(box)  # Convert to integer
            #print(contour)
            # Calculate the width and height of the rectangle
            width = rect[1][0]
            height = rect[1][1]

            # Ensure width is always smaller than height

            # Calculate the aspect ratio (length / width)
            aspect_ratio =  width/height if width > 0 else 0
            
            print(aspect_ratio)
            # Check if the aspect ratio matches the desired threshold
            if 0.5 <= aspect_ratio <= 0.9:  # Adjust the range if necessary
                # Draw the contour or rectangle on the result image
                cv2.drawContours(result, [box], -1, (0, 255, 0), 2)  # Green rectangle
                cv2.drawContours(result, [contour], -1, (0, 255, 0), -1)  # Fill the contour with green

    return result


In [30]:
import cv2
import numpy as np

def detect_hand_with_enhancements(image):
    # Step 1: Preprocessing (Blur to reduce noise)
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)

    # Step 2: Dynamic Skin-Tone Detection
    lower_skin = np.array([0, 30, 60], dtype=np.uint8)
    upper_skin = np.array([20, 150, 255], dtype=np.uint8)
    skin_mask = cv2.inRange(hsv, lower_skin, upper_skin)

    # Step 3: Adaptive Edge Detection
    gray = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY)
    gray = cv2.equalizeHist(gray)  # Enhance contrast
    edges = cv2.Canny(gray, 100, 200)

    # Ensure edges are connected
    kernel = cv2.getStructuringElement(cv2.MORPH_RECT, (5, 5))
    #edges = cv2.morphologyEx(edges, cv2.MORPH_CLOSE, kernel)

    # Step 4: Contour Detection on Skin Mask
    contours, _ = cv2.findContours(skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    filled_mask = np.zeros_like(edges)

    # Create a mask with additional padding for floodFill
    flood_fill_mask = np.zeros((edges.shape[0] + 2, edges.shape[1] + 2), dtype=np.uint8)
    flood_fill_mask[1:-1, 1:-1] = edges
    # Overlay for displaying centroids
    centroid_overlay = image.copy()

    # Skin tone mean for comparison
    skin_tone_mean = np.array([10, 100, 150])  # Mean skin tone in HSV
    threshold = 25  # Acceptable difference threshold

    for contour in contours:
        # Filter small contours
        if cv2.contourArea(contour) > 3000:
            # Calculate the centroid of the contour
            M = cv2.moments(contour)
            if M["m00"] != 0:  # Avoid division by zero
                x = int(M["m10"] / M["m00"])
                y = int(M["m01"] / M["m00"])

                # Draw the centroid point on the overlay
                cv2.circle(centroid_overlay, (x, y), 5, (0, 0, 255), -1)  # Red circle for centroid

                # Calculate the average color inside the contour
                mask = np.zeros(image.shape[:2], dtype=np.uint8)
                cv2.drawContours(mask, [contour], -1, 255, -1)  # Create a mask of the contour
                mean_color = cv2.mean(hsv, mask=mask)[:3]

                # Check if the mean color is within the threshold
                if np.all(np.abs(np.array(mean_color) - skin_tone_mean) < threshold):
                    # Flood-fill the region using the modified flood-fill mask
                    cv2.floodFill(filled_mask, flood_fill_mask, seedPoint=(x, y), newVal=255)

    # Step 5: Morphological Operations (Post-Processing)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    filled_mask = cv2.morphologyEx(filled_mask, cv2.MORPH_CLOSE, kernel)  # Close small gaps
    filled_mask = cv2.morphologyEx(filled_mask, cv2.MORPH_OPEN, kernel)   # Remove noise

    # Combine the filled regions with the original image
    result = cv2.bitwise_and(image, image, mask=filled_mask)

    return result, filled_mask, edges, centroid_overlay



In [31]:

def process_segments(image):
    # Convert the image to HSV and YCrCb color spaces
    blurred = cv2.GaussianBlur(image, (5, 5), 0)
    hsv = cv2.cvtColor(blurred, cv2.COLOR_BGR2HSV)
    ycrcb = cv2.cvtColor(blurred, cv2.COLOR_BGR2YCrCb)
    
    # Optimized thresholds for skin detection
    lower_hsv = np.array([0, 30, 60], dtype=np.uint8)
    upper_hsv = np.array([20, 150, 255], dtype=np.uint8)
    lower_ycrcb = np.array([0, 135, 85], dtype=np.uint8)
    upper_ycrcb = np.array([255, 180, 135], dtype=np.uint8)
    
    mask_hsv = cv2.inRange(hsv, lower_hsv, upper_hsv)
    mask_ycrcb = cv2.inRange(ycrcb, lower_ycrcb, upper_ycrcb)
    
    # Combine masks with weights
    combined_mask = cv2.addWeighted(mask_hsv, 0.5, mask_ycrcb, 0.5, 0)
    kernel = cv2.getStructuringElement(cv2.MORPH_ELLIPSE, (5, 5))
    skin_mask = cv2.morphologyEx(combined_mask, cv2.MORPH_OPEN, kernel)
    skin_mask = cv2.morphologyEx(skin_mask, cv2.MORPH_CLOSE, kernel)
    
    # Find contours in the mask
    contours, _ = cv2.findContours(skin_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = [cnt for cnt in contours if cv2.contourArea(cnt) > 3000]
    contours = sorted(contours, key=cv2.contourArea, reverse=True)

    segmented_image = image.copy()

    if len(contours) >= 2:
        largest = contours[0]
        second_largest = contours[1]
    elif len(contours) == 1:
        x, y, w, h = cv2.boundingRect(contours[0])
        top_half = contours[0][contours[0][:, 0, 1] < y + h // 2]
        bottom_half = contours[0][contours[0][:, 0, 1] >= y + h // 2]
        if top_half.shape[0] > 0 and bottom_half.shape[0] > 0:
            largest = top_half
            second_largest = bottom_half
        else:
            return image, None, None, None
    else:
        return image, None, None, None

    # Debugging: Print contour shapes
    print(f"Largest Contour: {largest.shape}")
    print(f"Second Largest Contour: {second_largest.shape}")

    # Draw red and blue regions on the segmented image
    if len(largest) > 0:
        cv2.drawContours(segmented_image, [largest], -1, (255, 0, 0), -1)  # Blue for the largest
    if len(second_largest) > 0:
        cv2.drawContours(segmented_image, [second_largest], -1, (0, 0, 255), -1)  # Red for the second largest

    def resize_with_aspect_ratio(image, target_size=(300, 300), pad_color=0):
        h, w = image.shape[:2]
        target_w, target_h = target_size
        scale = min(target_w / w, target_h / h)
        new_w, new_h = int(w * scale), int(h * scale)
        resized_image = cv2.resize(image, (new_w, new_h), interpolation=cv2.INTER_LINEAR)
        canvas = np.full((target_h, target_w), pad_color, dtype=np.uint8)
        x_offset = (target_w - new_w) // 2
        y_offset = (target_h - new_h) // 2
        canvas[y_offset:y_offset + new_h, x_offset:x_offset + new_w] = resized_image
        return canvas

    def create_segment(contour, target_size=(300, 300)):
        if contour is None or len(contour) == 0:
            return None
        mask = np.zeros_like(image[:, :, 0])
        cv2.drawContours(mask, [contour], -1, 255, -1)
        x, y, w, h = cv2.boundingRect(contour)
        cropped = mask[y:y + h, x:x + w]
        return resize_with_aspect_ratio(cropped, target_size)

    blue_segment = create_segment(largest)
    red_segment = create_segment(second_largest)

    return image, segmented_image, blue_segment, red_segment

def main():
    cap = cv2.VideoCapture(0)

    while True:
        ret, frame = cap.read()
        if not ret:
            break
        frame = cv2.flip(frame, 1)
        original, segmented, blue_part, red_part = process_segments(frame)

        if original is not None:
            cv2.imshow("Original Image", cv2.resize(original, (600, 400)))
        if segmented is not None:
            cv2.imshow("Red and Blue Screen", cv2.resize(segmented, (600, 400)))
        if blue_part is not None:
            cv2.imshow("Blue Part", blue_part)
        if red_part is not None:
            cv2.imshow("Red Part", red_part)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            break

    cap.release()
    cv2.destroyAllWindows()

if __name__ == "__main__":
    main()


Largest Contour: (157, 1, 2)
Second Largest Contour: (129, 1, 2)
Largest Contour: (160, 1, 2)
Second Largest Contour: (144, 1, 2)
Largest Contour: (143, 1, 2)
Second Largest Contour: (183, 1, 2)
Largest Contour: (146, 1, 2)
Second Largest Contour: (125, 1, 2)
Largest Contour: (161, 1, 2)
Second Largest Contour: (120, 1, 2)
Largest Contour: (150, 1, 2)
Second Largest Contour: (129, 1, 2)
Largest Contour: (144, 1, 2)
Second Largest Contour: (144, 1, 2)
Largest Contour: (162, 1, 2)
Second Largest Contour: (137, 1, 2)
Largest Contour: (170, 1, 2)
Second Largest Contour: (118, 1, 2)
Largest Contour: (141, 1, 2)
Second Largest Contour: (120, 1, 2)
Largest Contour: (164, 1, 2)
Second Largest Contour: (115, 1, 2)
Largest Contour: (148, 1, 2)
Second Largest Contour: (105, 1, 2)
Largest Contour: (148, 1, 2)
Second Largest Contour: (105, 1, 2)
Largest Contour: (154, 1, 2)
Second Largest Contour: (114, 1, 2)
Largest Contour: (149, 1, 2)
Second Largest Contour: (111, 1, 2)
Largest Contour: (138, 1,